In [140]:
from metar import Metar
from datetime import datetime, timedelta
from metar import Metar
from util.utils import *
import pandas as pd
import re


In [141]:

def interpretaMetaR(metarString):

    if("METAF" in metarString):
        metarString = metarString.replace('METAF', 'METAR')
        metarString = metarString.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarString[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet

def direction_to_degrees(df):
    direction_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': 0,
    'variable': 0,
    }

    df['windDirection'] = df['windDirection'].map(direction_degrees)
    return df

def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df

def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final

def sort_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['aero','hora_formatted'])

    return df
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []
    non_numeric_columns = []

    for coluna in colunas:
        if pd.api.types.is_numeric_dtype(df[coluna]):
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
            df_resampled_temp = df[coluna].resample('30T').mean()
            df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
            tempSeries = calculate_average(df_resampled_temp_filled)
            listSeries.append(tempSeries)
        else:
            non_numeric_columns.append(coluna)

    dataframe_resultante = pd.concat(listSeries, axis=1)

    # Criar um novo DataFrame para armazenar as colunas não numéricas
    df_nao_numericas = df[non_numeric_columns]

    # Redefinir os índices dos DataFrames antes da concatenação
    dataframe_resultante.reset_index(drop=True, inplace=True)
    df_nao_numericas.reset_index(drop=True, inplace=True)

    # Concatenar o DataFrame resultante com o DataFrame das colunas não numéricas
    resultado_final = pd.concat([dataframe_resultante, df_nao_numericas], axis=1)

    return resultado_final

def read_metar(tabela):
    if (tabela == "metar"):
        df = openDataframe("metar_bronze.csv")
        resultado_df =  df['metar'].apply(interpretaMetaR)
    elif (tabela == "metaf"):
        df = openDataframe("metaf_bronze.csv")
        resultado_df =  df['metaf'].apply(interpretaMetaR)

    resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['station', 'type', 'metar_hora_formated', 'station','dew_point','windDirection','windVelocity','visibility','pressure','weather','sky','skyFeet'] )
    df_bronze = pd.concat([df, resultado_df], axis=1)
    df_bronze = direction_to_degrees(df_bronze)
    df_bronze = sort_df(df_bronze)
    df_bronze = df_bronze.reset_index(drop=True)
    df_bronze['hora_formatted'] = pd.to_datetime(df_bronze['hora_formatted'])
    df_bronze.set_index(['aero','hora_formatted'], inplace=True)
    return df_bronze
    df_media = pipemediaTempoMetar(df_bronze,['hora','aero','dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
    df_no_columns = df_bronze.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)
    return df_media,df_bronze



In [142]:
df = read_metar("metaf")

In [143]:
df.sort_index().head(15)

hora  \
aero hora_formatted                       
SBBR 2023-03-20 03:00:00  1679292000000   
     2023-03-20 04:00:00  1679295600000   
     2023-03-20 05:00:00  1679299200000   
     2023-03-20 06:00:00  1679302800000   
     2023-03-20 07:00:00  1679306400000   
     2023-03-21 03:00:00  1679378400000   
     2023-03-21 04:00:00  1679382000000   
     2023-03-21 05:00:00  1679385600000   
     2023-03-21 06:00:00  1679389200000   
     2023-03-21 07:00:00  1679392800000   
     2023-03-21 23:00:00  1679450400000   
     2023-03-22 05:00:00  1679472000000   
     2023-03-22 06:00:00  1679475600000   
     2023-03-22 07:00:00  1679479200000   
     2023-03-23 03:00:00  1679551200000   

                                                                      metaf  \
aero hora_formatted                                                           
SBBR 2023-03-20 03:00:00  METAF SBBR 200600Z  32003KT 3000     BR SCT020...   
     2023-03-20 04:00:00  METAF SBBR 200700Z 01001KT 4000     BR SCT018 ...   
     2023-03-20 05:00:00  METAF SBBR 200800Z  14002KT 3000     BR BKN017...   
     2023-03-20 06:00:00  METAF SBBR 200900Z  16002KT 2000     BR BKN019...   
     2023-03-20 07:00:00  METAF SBBR 201000Z  16001KT 2000     BR BKN015...   
     2023-03-21 03:00:00  METAF SBBR 210600Z  10002KT 1000     BR OVC027...   
     2023-03-21 04:00:00  METAF SBBR 210700Z  11002KT 1000     BR OVC023...   
     2023-03-21 05:00:00  METAF SBBR 210800Z  12002KT 1000     BR BKN015...   
     2023-03-21 06:00:00  METAF SBBR 210900Z  10002KT 1000     BR BKN016...   
     2023-03-21 07:00:00  METAF SBBR 211000Z  12003KT 2000     BR BKN018...   
     2023-03-21 23:00:00  METAF SBBR 220200Z  35003KT 4000    RABR SCT01...   
     2023-03-22 05:00:00  METAF SBBR 220800Z  13003KT 5000     BR FEW031...   
     2023-03-22 06:00:00  METAF SBBR 220900Z  15002KT 3000     BR BKN033...   
     2023-03-22 07:00:00  METAF SBBR 221000Z  15002KT 4000     BR BKN033...   
     2023-03-23 03:00:00  METAF SBBR 230600Z  12002KT 4000     BR SCT028...   

                         station                                         type  \
aero hora_formatted                                                             
SBBR 2023-03-20 03:00:00    SBBR   routine report, cycle 6 (automatic report)   
     2023-03-20 04:00:00    SBBR   routine report, cycle 7 (automatic report)   
     2023-03-20 05:00:00    SBBR   routine report, cycle 8 (automatic report)   
     2023-03-20 06:00:00    SBBR   routine report, cycle 9 (automatic report)   
     2023-03-20 07:00:00    SBBR  routine report, cycle 10 (automatic report)   
     2023-03-21 03:00:00    SBBR   routine report, cycle 6 (automatic report)   
     2023-03-21 04:00:00    SBBR   routine report, cycle 7 (automatic report)   
     2023-03-21 05:00:00    SBBR   routine report, cycle 8 (automatic report)   
     2023-03-21 06:00:00    SBBR   routine report, cycle 9 (automatic report)   
     2023-03-21 07:00:00    SBBR  routine report, cycle 10 (automatic report)   
     2023-03-21 23:00:00    SBBR   routine report, cycle 2 (automatic report)   
     2023-03-22 05:00:00    SBBR   routine report, cycle 8 (automatic report)   
     2023-03-22 06:00:00    SBBR   routine report, cycle 9 (automatic report)   
     2023-03-22 07:00:00    SBBR  routine report, cycle 10 (automatic report)   
     2023-03-23 03:00:00    SBBR   routine report, cycle 6 (automatic report)   

                               metar_hora_formated station dew_point  \
aero hora_formatted                                                    
SBBR 2023-03-20 03:00:00  Wed Sep 20 03:00:00 2023    18.0      16.0   
     2023-03-20 04:00:00  Wed Sep 20 04:00:00 2023    18.0      16.0   
     2023-03-20 05:00:00  Wed Sep 20 05:00:00 2023    18.0      16.0   
     2023-03-20 06:00:00  Wed Sep 20 06:00:00 2023    17.0      16.0   
     2023-03-20 07:00:00  Wed Sep 20 07:00:00 2023    17.0      16.0   
     2023-03-21 03:00:00  Thu Sep 21 03:00:00 2023    17.0      16.0   
     